# Reconhecimento de Faces

Inicialmente, os imports:

In [2]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from time import time
from numba import njit, prange

E algumas funções auxiliares:

In [3]:
def to_row_vec(a, r=112, c=92):
    return np.reshape( a, (1, r*c), order='C')

def to_array(v, r=112, c=92):
    return np.reshape( v, (r,c) )

Vamos ler as imagens e colocá-las numa matriz de dados:

In [4]:
PATH = "/home/thelmo/uece/cursos/analise_de_dados/"
PATH = PATH+"jupyter/datasets/orl-faces/s{:d}/{:d}.pgm"

# ORL images are 112x92
rows = 112
cols = 92
m = rows * cols
ind = 40 # number of individuals
pic = 10 # number of images per individual

X_orig = np.empty( (1,m) )
for d in range(1,ind+1):
    for f in range(1,pic+1):
        im_file = PATH.format(d, f)
        img = to_row_vec( np.array( mpimg.imread(im_file) / 255.0 ) )
        X_orig = np.vstack( (X_orig, img) )
X_orig = np.delete(X_orig, 0, 0) # remove first row of zeros

# Create array with train and test data
X_train = np.copy(X_orig)
X_test = np.zeros((ind,m))
y_train = np.zeros((pic*ind,), dtype=int)
y_test = np.zeros((ind,), dtype=int)
for i in range(ind):
    X_test[i,:] = X_orig[pic*i,:]
    y_test[i] = i + 1
    y_train[pic*i:pic*(i+1)] = i + 1 
for i in range(ind-1,-1,-1):
    X_train = np.delete(X_train, pic*i, axis=0)
    y_train = np.delete(y_train, pic*i)
print('Dimensões: \nX_train: ', X_train.shape, \
      '   y_train:', y_train.shape, \
      '\nX_test:  ', X_test.shape, \
      '    y_test:  ', y_test.shape)
ntrains = X_train.shape[0]
ntests = X_test.shape[0]

# Remove mean
mu_train = np.mean(X_train, axis=1)
X_train = ( X_train.T - mu_train ).T
mu_test = np.mean(X_test, axis=1)
X_test = ( X_test.T - mu_test ).T

Dimensões: 
X_train:  (360, 10304)    y_train: (360,) 
X_test:   (40, 10304)     y_test:   (40,)


Calculemos a Decomposição de Valor Singular de $X_{train}$.<br>
Atenção para as dimensões da decompoisção reduzida:<br>
$$
\begin{array}{ccccc}
X & = & U & S & V^T \\
(360 \times 10304) & & (360 \times 360) & (360 \times 360) 
& (360 \times 10304) 
\end{array}
\, .
$$
A matriz $V$, que é igual à matriz dos autovetores da matriz de covariância dos dados $Q$,<br>
está sem as colunas correspondentes aos autovetores nulos.

In [25]:
# Calculate SVD of X_train
U, S, Vt = np.linalg.svd(X_train, full_matrices=False)
print('Dimensões: U: ', U.shape, ', S: ', S.shape, ', Vt: ', Vt.shape)
energy = np.cumsum(S**2) / np.sum(S**2)

thres = 0.95 #0.83 # 0.95 # 0.80

idx = [ n for n, i in enumerate(energy) if i > thres ][0]
print( 'Variabilidade acumulada: {:.2f}%, até o {:d}-ésimo autovalor.'\
      .format(100 * energy[idx], idx) )

# w, Q = np.linalg.eigh(np.cov(X_train, rowvar=False))
# print(w.shape, Q.shape)

Dimensões: U:  (360, 360) , S:  (360,) , Vt:  (360, 10304)
Variabilidade acumulada: 95.03%, até o 136-ésimo autovalor.


Estas são as 20 primeiras <i>eigenfaces</i>:

In [31]:
fig = plt.figure(figsize=(10, 5))
fig.suptitle('As 20 Primeiras Eigenfaces')

for i in range(20):
    sub = fig.add_subplot(4, 5, i + 1)
    sub.imshow(to_array(Vt[i,:]), cmap='gray')
    sub.set_xticks([])
    sub.set_yticks([])
    sub.set_xticklabels([])
    sub.set_yticklabels([])
plt.show()

Façamos a redução de dimensionalidade propriamente dita.

In [18]:
# idx = 28
V_hat  = Vt[:idx,:].T # Cuidado: V e não Vt
# Y_train = U[:,:idx] * S[:idx] # Y = X V = U S Vt V = U S
Y_train = np.dot( X_train, V_hat )
# print(np.max(np.abs(Y_train - Y_train2)))
Y_test  = np.dot( X_test,  V_hat )

print( 'Dimensões: \nX_train: {:} --> Y_train:  {:}'\
      .format(X_train.shape, Y_train.shape) )
print( 'X_test:   {:} --> Y_test:    {:}'\
      .format(X_test.shape, Y_test.shape) )   
print( 'V:       {:} --> V_hat: {:}'\
      .format(Vt.T.shape, V_hat.shape) )

Dimensões: 
X_train: (360, 10304) --> Y_train:  (360, 28)
X_test:   (40, 10304) --> Y_test:    (40, 28)
V:       (10304, 360) --> V_hat: (10304, 28)


Um teste inicial para calcular a acurácia.

In [19]:
error_count = 0
error_ids = []
for i in range(ntests):
    dist = []
    for k in range(ntrains):
        dist = np.append(dist, np.linalg.norm(Y_test[i,:] - Y_train[k,:]))
    index = np.argmin(dist)
    if y_train[index]!=y_test[i]:
#     if index//9 != i:
        error_count += 1
        error_ids.append(i+1)
print( 'Número de erros: {:d} (id# {:}). \nAcurácia: {:.2f}%.'\
      .format(error_count, error_ids, 100 * (ind - error_count) / ind) )

Número de erros: 1 (id# [35]). 
Acurácia: 97.50%.


Agora, procedemos com o reconhecimento propriamente dito.

In [26]:
# Select new test image to be recognized,
# from 1 to 40 (errors: 1 and 35).
new_face = 2

# Restore means
X_train_show = (X_train.T + mu_train).T
X_test_show  = (X_test.T  + mu_test).T

# Esta tentativa faz os cálculos ANTES de mostrar as imagens.
# O range do loop abaixo deve ser range(ntrains+1)
#dist = []
#for k in range(ntrains):
#    dist = np.append(dist, np.linalg.norm(Y_test[new_face-1,:] - Y_train[k,:]))
#index = np.argmin(dist)
#print('Identidade real:     ', new_face, \
#      '\nIdentidade estimada: ', index//9 + 1, '(index =', index, '\b)')
# Append result image to X_train_show
#X_train_show = np.vstack( (X_train_show, X_train_show[index,:]) )

fig = plt.figure(figsize=(8, 4))
ax1 = fig.add_subplot(1,2,1)
ax1.imshow(to_array(X_test_show[new_face-1,:]), cmap='gray')
ax1.set_title('Nova Imagem (Teste)')
ax1.set_xticks([])
ax1.set_yticks([])
ax1.set_xticklabels([])
ax1.set_yticklabels([])
ax2 = fig.add_subplot(1,2,2)
ax2.set_title('Imagem Mais Similar')
ax2.set_xticks([])
ax2.set_yticks([])
ax2.set_xticklabels([])
ax2.set_yticklabels([])

dist = []
im = plt.imshow(np.random.randn(rows,cols), vmin=0, vmax=1, cmap='gray')
plt.ion()
for k in range(ntrains):
    dist = np.append(dist, np.linalg.norm(Y_test[new_face-1,:] - Y_train[k,:]))
    A = to_array(X_train_show[k,:])
    im.set_data(A)
    plt.pause(0.005)
index = np.argmin(dist)
print('Identidade real:     ', new_face, \
      '\nIdentidade estimada: ', y_train[index], \
      '(índice =', index, '\b)')
im.set_data( to_array(X_train_show[index,:]) )
plt.pause(0.005)
plt.ioff()
#plt.show()

Identidade real:      2 
Identidade estimada:  2 (índice = 10 )


Vejamos as imagens usadas pelo computador na busca.

In [27]:
# Select new test image to be recognized,
# from 1 to 40 (errors: 1 and 35).
new_face = 2

# Reshape reduced images
Y_train_show = np.dot( U[:,0:idx] * S[0:idx], Vt[0:idx,:] )
Y_test_show  = np.dot( Y_test, V_hat.T )

# Restore means
X_train_show = (X_train.T + mu_train).T
X_test_show  = (X_test.T  + mu_test).T
Y_train_show = (Y_train_show.T + mu_train).T
Y_test_show  = (Y_test_show.T + mu_test).T

fig = plt.figure(figsize=(12, 4))
ax1 = fig.add_subplot(1,3,1)
ax1.imshow(to_array(X_test[new_face-1,:]), cmap='gray')
ax1.set_title('Nova Imagem (Teste)')
ax1.set_xticks([])
ax1.set_yticks([])
ax1.set_xticklabels([])
ax1.set_yticklabels([])
ax2 = fig.add_subplot(1,3,2)
ax2.imshow(to_array(Y_test_show[new_face-1,:]), cmap='gray')
ax2.set_title('Nova Imagem (Teste)')
ax2.set_xticks([])
ax2.set_yticks([])
ax2.set_xticklabels([])
ax2.set_yticklabels([])
ax3 = fig.add_subplot(1,3,3)
ax3.set_title('Imagem Mais Similar')
ax3.set_xticks([])
ax3.set_yticks([])
ax3.set_xticklabels([])
ax3.set_yticklabels([])

dist = []
im = plt.imshow(np.random.randn(rows,cols), vmin=0, vmax=1, cmap='gray')
plt.ion()
for k in range(ntrains):
    dist = np.append(dist, np.linalg.norm(Y_test[new_face-1,:] - Y_train[k,:]))
    A = to_array(Y_train_show[k,:])
    im.set_data(A)
    plt.pause(0.005)
index = np.argmin(dist)
print('Identidade real:     ', new_face, \
      '\nIdentidade estimada: ', y_train[index], '(índice =', index, '\b)')
im.set_data( to_array(Y_train_show[index,:]) )
plt.pause(0.005)
plt.ioff()
#plt.show()

Identidade real:      2 
Identidade estimada:  2 (índice = 10 )


Verifiquemos a acurácia desta execução.

In [15]:
error_count = 0
error_ids = []
for i in range(ntests):
    dist = []
    for k in range(ntrains):
        dist = np.append(dist, np.linalg.norm(Y_test[i,:] - Y_train[k,:]))
    index = np.argmin(dist)
    if y_train[index]!=y_test[i]:
        error_count += 1
        error_ids.append(i+1)
print( 'Número de erros: {:d} (id# {:}). \nAcurácia: {:.2f}%.'\
      .format(error_count, error_ids, 100 * (ind - error_count) / ind) )

Número de erros: 2 (id# [1, 35]). 
Acurácia: 95.00%.


OFFLINE: Podemos gerar os gráficos de acurácia.

In [16]:
def principal_components(r): # Y = X V = U S Vt V = U S
    # Calcula as componentes principais:
    V_hat  = Vt[:r,:].T # Cuidado: V e não Vt
    Ytrain = U[:,:r] * S[:r] # Y = X V = U S Vt V = U S
    Ytest  = np.dot( X_test,  V_hat )
    return Ytrain, Ytest

start_time = time()

@njit(fastmath=True, cache=True)
def find_match(Ytrain, Ytest, ytrain, ytest, ntrains, ntests):
    count = 0
    for i in range(ntests):
        min_dist = 10000
        for j in range(ntrains):
            dist = np.linalg.norm(Ytest[i,:] - Ytrain[j,:])
            if dist <= min_dist:
                index = j
                min_dist = dist
        if ytrain[index] == ytest[i]:
            count += 1
    return count / ntests
           
ntrains = 360
ntests  = 40

acc = []
for r in range(1,360):
    Y_train, Y_test = principal_components(r)
    acc = np.append( acc, \
        find_match(Y_train, Y_test, y_train, y_test, ntrains, ntests) )

elapsed = time() - start_time
PATH = '/home/thelmo/uece/cursos/analise_de_dados/jupyter/'
np.save(PATH+'orl_acc.npy', acc)

print('Tempo de execução: {:6.2f} s.'.format(elapsed) )

Tempo de execução:   3.75 s.


O gráfico a seguir aprensenta as acurácias obtidas para cada quantidade de autovetores retidos.

In [32]:
acc = np.load(PATH+'orl_acc.npy')

fig = plt.figure(figsize=(12, 4))

ax1 = fig.add_subplot(1,2,1)
ax1.set_title('Acurácia por Variabilidade Acumulada')
ax1.set_xlabel('Números de Autovetores')
ax1.set_ylabel('Acurácia (%)')
ax1.plot(list(range(1,359)), acc[1:359])

ax2 = fig.add_subplot(1,2,2)
ax2.set_title('Acurácia por Variabilidade Acumulada - Detalhe')
ax2.set_xlabel('Números de Autovetores')
ax2.set_ylabel('Acurácia (%)')
ax2.plot(list(range(10,60)), acc[10:60])

plt.show()

<b>NEM SEMPRE MAIS INFORMAÇÃO É MELHOR!</b>